In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio
import cv2

import mediapipe as mp

In [2]:
dataset_path = os.listdir('dataset/train')
label_types = os.listdir('dataset/train')

In [3]:
# 훈련 데이터셋을 위한 비어있는 리스트 초기화
rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/train' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('dataset/train'+'/'+item)    
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('dataset/train'+'/'+item)+'/'+room))
# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name']).loc[:,['video_name','tag']]
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('train.csv', encoding='utf-8-sig')

# 테스트 데이터셋 경로 설정
dataset_path = os.listdir('dataset/test')

# 테스트 데이터셋의 방 유형 리스트 생성
room_types = os.listdir('dataset/test')

# 테스트 데이터셋을 위한 비어있는 리스트 초기화
rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/test' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('dataset/test'+'/'+item)
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('dataset/test'+'/'+item)+'/'+room))

# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name'])
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('test.csv', encoding='utf-8-sig')

# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
# GPU가 사용 가능한 경우
if gpus:
    try:
        # 첫 번째 GPU에 대해 메모리 제한 설정
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], 
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]
        )
    except RuntimeError as e:
        print(e)

# 훈련 데이터셋과 테스트 데이터셋을 각각 CSV 파일에서 로드
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 훈련 및 테스트 데이터셋 크기 출력
print(f"Total video for training: {len(train_df)}")
print(f"Total video for testing: {len(test_df)}")

# 훈련 데이터셋의 샘플 5개 출력
train_df.sample(5)

# 이미지 크기를 224로 설정
IMG_SIZE = 224

mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

Total video for training: 10
Total video for testing: 10


In [4]:
# 주어진 이미지에서 중앙에 맞춰 정사각형으로 잘나내는 함수
def crop_center_square(frame):
    # 이미지의 높이(y)와 너비(x)를 가져옴
    y, x = frame.shape[0:2]
    # 이미지의 높이와 너비 중 더 작은 값을 선택하여 정사각형의 크기를 결정
    min_dim = min(y, x)
    # 정사각형을 이미지 중앙에 위치시키기 위해 시작점의 x좌표와 y좌표를 계산
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    # 계산된 시작점과 정사각형의 크기를 이용하여 이미지의 중앙 부분을 잘라냅니다.
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

# 비디오 파일을 로드하고, 각 프레임을 처리하여 배열로 반환하는 함수
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    # OpenCV를 사용하여 비디오 파일 열기
    cap = cv2.VideoCapture(path)
    frames = []
    skeletons = []  # 스켈레톤 데이터
    hand_landmarks = []  # 손 데이터
    face_landmarks = []  # 얼굴 데이터
    try:
        while True:
            # 비디오에서 프레임을 하나씩 읽기
            ret, frame = cap.read()
            # 읽을 프레임이 없으면 반복문을 종료
            if not ret:
                break
            # 읽은 프레임에서 중앙의 정사각형 부분을 잘라냄
            frame = crop_center_square(frame)
            # 프레임의 크기를 지정된 크기로 조절
            frame = cv2.resize(frame, resize)            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Mediapipe를 사용하여 스켈레톤 추출
            hands_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)
            face_results = face_mesh.process(frame_rgb)
           
            if pose_results.pose_landmarks:
                skeletons.append(pose_results.pose_landmarks.landmark)
            if hands_results.multi_hand_landmarks:
                hand_landmarks.append(hands_results.multi_hand_landmarks)
            if face_results.detections:
                face_landmarks.append(face_results.detections)
            
            # OpenCV는 BGR 색상 순서를 사용하므로, 이를 RGB 순서로 변경
            frame = frame[:, :, [2, 1, 0]]
            # 처리된 프레임을 프레임 리스트에 추가
            frames.append(frame)
            # max_frames가 지정된 경우, 지정된 수의 프레임만큼만 처리
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 파일을 닫기
        cap.release()
        pose.close()
        hands.close()
        face_detection.close()
    # 처리된 모든 프레임을 numpy 배열로 변환하여 반환
    return np.array(frames), skeletons, hand_landmarks, face_landmarks

def build_feature_extractor():
    # InceptionV3 모델을 초기화, ImageNet 데이터셋으로 사전 훈련된 가중치를 사용
    # 모델의 최상위 층은 제외, 출력으로 평균 풀링을 사용
    # input_shape는 이미지의 크기를 정의
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),)
    # InceptionV3에 맞는 입력 데이터 전처리 함수를 할당
    preprocess_input = keras.applications.inception_v3.preprocess_input    
    # 입력 텐서를 정의
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    # 전처리 함수를 이용해 입력 데이터를 전처리
    preprocessed = preprocess_input(inputs)    
    # 전처리된 데이터를 특징 추출 모델에 통과시켜 출력
    outputs = feature_extractor(preprocessed)
    # 입력과 출력을 연결하는 새로운 Keras 모델을 정의하고 반환
    return keras.Model(inputs, outputs, name="feature_extractor")

# 데이터프레임에서 비디오 데이터를 처리하고 결과 반환하는 함수
def prepare_all_video(df, root_dir):
    # 데이터프레임에 있는 샘플의 수를 계산
    num_samples = len(df)
    # 비디오 파일의 경로를 리스트로 추출
    video_paths = df["video_name"].values.tolist()
    # 레이블을 numpy 배열로 변환
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # 각 비디오에 대해 처리할 최대 프레임 수를 저장하는 마스크 배열을 초기화
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")

    # 비디오에서 추출한 특징을 저장할 배열을 초기화
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
        
    for idx, path in enumerate(video_paths):
        cap = cv2.VideoCapture(os.path.join(root_dir, path))
        frames_processed = 0

        while True:
            ret, frame = cap.read()
            if not ret or frames_processed == MAX_SEQ_LENGTH:
                break

            # 프레임 처리
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Mediapipe로 손과 포즈 데이터 추출
            hand_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)

            # TODO: 추출된 hand_results와 pose_results를 사용하여 프레임의 특징을 추출하는 로직 추가

            frame_masks[idx, frames_processed] = 1
            frames_processed += 1

        cap.release()

    pose.close
    hands.close

    return (frame_features, frame_masks), labels

# RNN모델 구축하고 반환하는 함수
def get_sequence_model():
    # 레이블 처리기에서 클래스 어휘 사전을 가져오기
    class_vocab = label_processor.get_vocabulary()
    # 입력 텐서들을 정의. 하나는 프레임 특징, 다른 하나는 마스크
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")
    # GRU(Gated Recurrent Unit) 레이어를 사용하여 시퀀스 데이터를 처리
    # return_sequences=True는 중간 출력을 모두 반환하도록 설정
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)

    # 또 다른 GRU 레이어를 추가.이번에는 중간 출력을 반환하지 않음
    x = keras.layers.GRU(8)(x)
    # 드롭아웃 레이어를 추가하여 과적합을 방지
    x = keras.layers.Dropout(0.4)(x)
    # 밀집 연결(Dense) 레이어를 추가.
    x = keras.layers.Dense(8, activation="relu")(x)

    # 최종 출력 레이어를 추가.클래스 수에 맞춰 출력 노드를 설정하고 소프트맥스 활성화 함수를 사용
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)
    # 정의된 입력과 출력을 사용하여 새로운 Keras 모델을 생성
    rnn_model = keras.Model([frame_features_input, mask_input], output)

    # 모델을 컴파일. 손실 함수'sparse_categorical_crossentropy',옵티마이저 'adam'을 사용
    rnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return rnn_model

# 비디오 분류모델을 훈련하고 평가하는 함수
def run_experiment():
    # 모델 체크포인트를 저장할 파일 경로 설정
    filepath = "./tmp/video_classifier.h5"
    # 훈련 중에 가장 좋은 모델의 가중치만 저장 - ModelCheckpoint 콜백을 생성
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1)
    
    # get_sequence_model 함수를 사용하여 순환 신경망 모델을 생성
    seq_model = get_sequence_model()
    # 모델을 훈련 데이터로 학습. 검증 데이터는 훈련 데이터의 30%를 사용
    history = seq_model.fit(
        [train_data[0], train_data[1]],  # 훈련 데이터: 프레임 특징과 마스크
        train_labels,                    # 훈련 레이블
        validation_split=0.3,            # 검증 데이터 분할 비율
        epochs=EPOCHS,                   # 에폭 수
        callbacks=[checkpoint],          # 콜백 (체크포인트)
    )    
    # 가장 좋은 모델의 가중치 불러오기
    seq_model.load_weights(filepath)

    # 테스트 데이터로 모델을 평가하고 정확도를 출력
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"정확도: {round(accuracy * 100, 2)}%")   
    
    return history, seq_model

feature_extractor = build_feature_extractor()

label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[...,None]).numpy()

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048


# 단일 비디오 프레임을 준수하는 함수
def prepare_single_video(frame, max_seq_length=MAX_SEQ_LENGTH, num_features=NUM_FEATURES):
    # 프레임 차원 추가: (높이, 너비, 채널) -> (1, 높이, 너비, 채널)
    frame = np.expand_dims(frame, axis=0)
    # 프레임을 모델에 입력할 수 있는 형식으로 전처리
    preprocessed_frame = keras.applications.inception_v3.preprocess_input(frame)
    # 모델을 사용하여 프레임의 특징을 추출
    features = feature_extractor.predict(preprocessed_frame)
    
    # 추출된 특징을 포함하는 배열을 초기화
    frame_features = np.zeros(shape=(1, max_seq_length, num_features), dtype="float32")
    # 첫 번째 위치에 추출된 특징을 저장
    frame_features[0, 0, :] = features
    
    # 첫 번째 위치의 마스크를 True로 설정
    frame_mask = np.zeros(shape=(1, max_seq_length), dtype="bool")
    frame_mask[0, 0] = True
    
    return frame_features, frame_mask

# 비디오 에측 하는 함수
def sequence_prediction(path):    
    class_vocab = label_processor.get_vocabulary()

    # 지정된 경로에서 비디오를 로드
    frames = load_video(os.path.join("test", path))

    # 로드한 비디오 프레임에 대해 특징 추출과 마스크 준비
    frame_features, frame_mask = prepare_single_video(frames)

    # 순환 신경망 모델을 사용하여 예측
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    # 예측된 확률을 내림차순으로 정렬하여 가장 높은 확률을 가진 클래스를 출력
    for i in np.argsort(probabilities)[::-1]:
        print(f"{class_vocab[i]} : {probabilities[i]*100:5.2f}%")

    return frames

['가볍다', '가져오다', '가짜', '가치', '보관', '보내다', '보다', '안경', '알다', '월요일']


In [7]:
train_data, train_labels = prepare_all_video(train_df, "train")
test_data, test_labels = prepare_all_video(test_df,"test")

_, sequence_model = run_experiment()

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path : {test_video}")
predicted_frames = sequence_prediction(test_video)
print(predicted_frames)

Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 2.3026 - accuracy: 0.1429
Epoch 1: val_loss improved from inf to 2.30399, saving model to ./tmp\video_classifier.h5
1/1 [==============================] - 15s 15s/step - loss: 2.3026 - accuracy: 0.1429 - val_loss: 2.3040 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 2.3020 - accuracy: 0.1429
Epoch 2: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 54ms/step - loss: 2.3020 - accuracy: 0.1429 - val_loss: 2.3054 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 2.3014 - accuracy: 0.1429
Epoch 3: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 58ms/step - loss: 2.3014 - accuracy: 0.1429 - val_loss: 2.3068 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 2.3008 - accuracy: 0.1429
Epoch 4: val_loss did not improve from 2.30399
1

Epoch 31/100
1/1 [==============================] - ETA: 0s - loss: 2.2850 - accuracy: 0.1429
Epoch 31: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 49ms/step - loss: 2.2850 - accuracy: 0.1429 - val_loss: 2.3462 - val_accuracy: 0.0000e+00
Epoch 32/100
1/1 [==============================] - ETA: 0s - loss: 2.2845 - accuracy: 0.1429
Epoch 32: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 46ms/step - loss: 2.2845 - accuracy: 0.1429 - val_loss: 2.3476 - val_accuracy: 0.0000e+00
Epoch 33/100
1/1 [==============================] - ETA: 0s - loss: 2.2839 - accuracy: 0.1429
Epoch 33: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 50ms/step - loss: 2.2839 - accuracy: 0.1429 - val_loss: 2.3490 - val_accuracy: 0.0000e+00
Epoch 34/100
1/1 [==============================] - ETA: 0s - loss: 2.2833 - accuracy: 0.1429
Epoch 34: val_loss did not improve from 2.30399
1/1 [==============================]

Epoch 61/100
1/1 [==============================] - ETA: 0s - loss: 2.2684 - accuracy: 0.1429
Epoch 61: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 61ms/step - loss: 2.2684 - accuracy: 0.1429 - val_loss: 2.3885 - val_accuracy: 0.0000e+00
Epoch 62/100
1/1 [==============================] - ETA: 0s - loss: 2.2679 - accuracy: 0.1429
Epoch 62: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 49ms/step - loss: 2.2679 - accuracy: 0.1429 - val_loss: 2.3899 - val_accuracy: 0.0000e+00
Epoch 63/100
1/1 [==============================] - ETA: 0s - loss: 2.2674 - accuracy: 0.1429
Epoch 63: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 48ms/step - loss: 2.2674 - accuracy: 0.1429 - val_loss: 2.3913 - val_accuracy: 0.0000e+00
Epoch 64/100
1/1 [==============================] - ETA: 0s - loss: 2.2668 - accuracy: 0.1429
Epoch 64: val_loss did not improve from 2.30399
1/1 [==============================]

Epoch 91/100
1/1 [==============================] - ETA: 0s - loss: 2.2529 - accuracy: 0.1429
Epoch 91: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 57ms/step - loss: 2.2529 - accuracy: 0.1429 - val_loss: 2.4306 - val_accuracy: 0.0000e+00
Epoch 92/100
1/1 [==============================] - ETA: 0s - loss: 2.2524 - accuracy: 0.1429
Epoch 92: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 53ms/step - loss: 2.2524 - accuracy: 0.1429 - val_loss: 2.4320 - val_accuracy: 0.0000e+00
Epoch 93/100
1/1 [==============================] - ETA: 0s - loss: 2.2519 - accuracy: 0.1429
Epoch 93: val_loss did not improve from 2.30399
1/1 [==============================] - 0s 54ms/step - loss: 2.2519 - accuracy: 0.1429 - val_loss: 2.4334 - val_accuracy: 0.0000e+00
Epoch 94/100
1/1 [==============================] - ETA: 0s - loss: 2.2514 - accuracy: 0.1429
Epoch 94: val_loss did not improve from 2.30399
1/1 [==============================]

ValueError: in user code:

    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2357, in predict_step
        return self(x, training=False)
    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\SeeUSoon\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
    

In [10]:
# 이미 정의된 build_feature_extractor와 get_sequence_model 함수를 사용하여 모델을 생성합니다.
feature_extractor = build_feature_extractor()
sequence_model = get_sequence_model()

# 미디어 파이프에서 제공하는 드로잉 유틸리티와 모델을 사용하기 위한 인스턴스를 생성
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh

# 웹캠을 사용하기 위해 cv2.VideoCapture 객체를 초기화
cap = cv2.VideoCapture(0)

# Hands, Pose, FaceMesh 객체를 생성
with mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands, mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose, mp_face_mesh.FaceMesh(
            max_num_faces=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
    # 웹캠이 열려 있는 동안 무한 루프를 돌면서 프레임을 읽기
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue  # 읽기에 실패하면 다음 프레임으로 건너뛰기

        # 좌우 반전 및 BGR에서 RGB로 변환
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # 중앙을 잘라내기
        image = crop_center_square(image)
        # 크기 조절
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        # 변환된 이미지로 손을 감지
        hands_results = hands.process(image)
        # 변환된 이미지로 포즈를 감지
        pose_results = pose.process(image)
        # 변환된 이미지로 얼굴 랜드마크를 감지
        face_results = face_mesh.process(image)
        
        frame_features, frame_mask = prepare_single_video(image)
        probabilities = sequence_model.predict([frame_features, frame_mask])
                
        # 미디어파이프 결과를 그림
        # 손가락 인식 결과를 그림
        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        # 포즈 인식 결과를 그림
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # 얼굴 랜드마크 인식 결과를 그림
        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))

        # 다시 BGR로 이미지를 변환하여 OpenCV에서 사용
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        print(probabilities)
        # 처리된 이미지를 'image'라는 창에 표시합니다.
        cv2.imshow('image', image)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 사용이 끝난 후, 웹캠을 해제합니다.
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 3s 3s/step
[[0.10046576 0.10240855 0.10703503 0.09368748 0.09792869 0.09779553
  0.10177188 0.10034169 0.09728411 0.10128126]]
1/1 [==============================] - 0s 25ms/step
[[0.10181145 0.10319967 0.10445561 0.09689961 0.09642919 0.09848427
  0.10198955 0.09890218 0.09393205 0.1038964 ]]
1/1 [==============================] - 0s 22ms/step
[[0.1015104  0.1032148  0.10481443 0.09669576 0.09660892 0.09862401
  0.10173338 0.09894211 0.09390303 0.10395319]]
1/1 [==============================] - 0s 24ms/step
[[0.10175023 0.10323639 0.10460287 0.09682146 0.09644533 0.09854633
  0.10184358 0.0989363  0.093909   0.10390855]]
1/1 [==============================] - 0s 24ms/step
[[0.10187922 0.10343812 0.10459799 0.09675049 0.09623821 0.09845383
  0.10181223 0.09914445 0.09408011 0.10360528]]
1/1 [==============================] - 0s 19ms/step
[[0.1016285  0.10341643 0.10490566 0.09649369 0.09629349 0.09842348
  0.10177817 0.09924338 0.09417324 0.10364